In [2]:
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys

Using TensorFlow backend.


In [3]:
# テキストを読み込む
path = "./kazeno_matasaburo.txt"
bindata = open( path, "rb" ).read()
text = bindata.decode("sjis")

In [13]:
# 文字とインデックスの相互変換用の変数
chars = sorted( list(set(text)) )
char_indices = dict( (c,i) for i, c in enumerate(chars) )
indices_char = dict( (i,c) for i,c in enumerate(chars) )

In [14]:
# 入力文長
maxlen = 40

# 学習データを作るステップ数
step = 3

# 入力文
sentences = []

# 出力となる入力文の次の文字
next_chars = []

In [15]:
# 学習データを作る
for i in range(0,len(text)-maxlen, step):
    sentences.append( text[i:i+maxlen] )
    next_chars.append( text[i+maxlen] )

In [16]:
# 学習データをワンホットベクトル表現にする
X = np.zeros((len(sentences), maxlen, len(chars) ), dtype=np.bool )
y = np.zeros((len(sentences), len(chars) ), dtype=np.bool)

for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [18]:
# モデルを定義する
model = Sequential()
model.add( LSTM(128, input_shape=(maxlen, len(chars))) )
model.add( Dense(len(chars)) )
model.add( Activation("softmax") )
optimizer = RMSprop( lr=0.01)
model.compile( loss="categorical_crossentropy", optimizer=optimizer )

In [19]:
# 学習
for iteration in range(1,60):
    print()
    print("-" * 50)
    print("繰り返し回数：", iteration)
    model.fit(X, y, batch_size=128, epochs=1)


--------------------------------------------------
繰り返し回数： 1
Epoch 1/1
11022/11022 [==============================] - 9s 814us/step - loss: 4.5731

--------------------------------------------------
繰り返し回数： 2
Epoch 1/1
11022/11022 [==============================] - 8s 719us/step - loss: 3.6122

--------------------------------------------------
繰り返し回数： 3
Epoch 1/1
11022/11022 [==============================] - 8s 711us/step - loss: 3.2010

--------------------------------------------------
繰り返し回数： 4
Epoch 1/1
11022/11022 [==============================] - 8s 707us/step - loss: 2.8628

--------------------------------------------------
繰り返し回数： 5
Epoch 1/1
11022/11022 [==============================] - 8s 705us/step - loss: 2.5575

--------------------------------------------------
繰り返し回数： 6
Epoch 1/1
11022/11022 [==============================] - 8s 715us/step - loss: 2.2494

--------------------------------------------------
繰り返し回数： 7
Epoch 1/1
11022/11022 [===========================

11022/11022 [==============================] - 8s 716us/step - loss: 0.1719

--------------------------------------------------
繰り返し回数： 57
Epoch 1/1
11022/11022 [==============================] - 8s 700us/step - loss: 0.1520

--------------------------------------------------
繰り返し回数： 58
Epoch 1/1
11022/11022 [==============================] - 8s 703us/step - loss: 0.1535

--------------------------------------------------
繰り返し回数： 59
Epoch 1/1
11022/11022 [==============================] - 8s 699us/step - loss: 0.1561


In [29]:
def sample(preds, temperture=1.0 ):
    preds = np.asarray( preds ).astype("float64")
    #preds = np.log(preds)/temperture
    #exp_preds = np.exp(preds)
    #preds = exp_preds / np.sum(exp_preds)

    # temputureによって確率を調整（温度が高い→ランダムな文が出てくる）
    preds = preds ** (1.0/temperture)
    
    # 正規化
    preds = preds / np.sum(preds)
    
    # 確率に従って1文字選択
    probs = np.random.multinomial(1, preds, 1)
    return np.argmax(probs)

start_index = random.randint(0,len(text)-maxlen-1)
sentence = text[start_index:start_index+maxlen]
generated = sentence

print( "「" + sentence + "」" + '  の続きを生成：' )
print("---------------------------")

sys.stdout.write( generated )

# 400文字生成
for i in range(400):
    
    # sentenceをワンホットベクトル表現に変換
    x = np.zeros( (1,maxlen, len(chars)) )
    for t, char in enumerate(sentence):
        x[0, t, char_indices[char]] = 1

    # 学習したモデルを使って予測
    prediction = model.predict( x, verbose=0 )[0]
    
    # 次の文字を確率的に選択
    next_index = sample( prediction, 1.0 )
    next_char = indices_char[next_index]

    # 生成された文字を追加して，後ろの40文字を次の入力にする
    generated += next_char
    sentence = sentence[1:] + next_char

    sys.stdout.write( next_char )
    sys.stdout.flush()

print()
        
    

「いいと箇条をたてていってごらん。そら。」三郎は先生みたいな顔つきをして指を一本だ」  の続きを生成：
---------------------------
いいと箇条をたてていってごらん。そら。」三郎は先生みたいな顔つきをして指を一本だもってしまうました。
　
　先生はだまっで秋して、しんと鳴りました。
　がんだか黒いでし。」
「うんけれど　どっく戻こうでした。みんなのするのでってやってしまいました。
「ないな顔だ、みんな女はきて砂がやみんなが運動場はめいでごのこままだすやらいのでした。
　ほんはうぐそのうしてから変てやっていているのでした。
　みんなは悦治《やんだぞ。食ぐ。」出したもみんで良をぶさがが横ままでがって一あそのうしろのを見ないでした。
　けれどもやったくりでした。
　耕助は何かかけはなりふり近のと手をからえながそこを出しばらく　栗《やり》をすっつけだぞ。あるがった。」そしてからみんなでした。
　みんなはこんの木の中をはているやとおうら出し行きようにしていますし、
　嘉助はょっと耕助をとうに三郎が、吉きました耕なからするのでもからもいいねったいといって来たのでした。
　みんなはだんだん
